# Training Regression - Reaction

# Import packages

In [60]:
import os
import sys

current_path=os.getcwd()
print(current_path)

parent_path=os.path.dirname(current_path)
print(parent_path)

if parent_path not in sys.path:
    sys.path.append(parent_path)

/home/labhhc2/Documents/workspace/D20/Tam/backup/chemprop_1/examples
/home/labhhc2/Documents/workspace/D20/Tam/backup/chemprop_1


In [61]:
import pandas as pd
from lightning import pytorch as pl
from pathlib import Path

from chemprop import data, featurizers, models, nn

# Change data inputs here

## Load data

In [62]:
import numpy as np
chemprop_dir = Path.cwd().parent
num_workers = 0  # number of workers for dataloader. 0 means using main process for data loading
# smiles_column = 'AAM'
# target_columns = ['lograte']

In [63]:
# train_path = chemprop_dir / "tests" / "data" / "regression" / "rxn" / "e2sn2" / "fold_4" / "aam_train.csv"
# train_npz = np.load(f'../chemprop/data/super/e2sn2/fold_4/e2sn2_aam_train_processed_data.npz', allow_pickle=True)
# train_v = train_npz['node_attrs']
# train_e = train_npz['edge_attrs']
# train_idx_g = train_npz['edge_indices']
# train_y = train_npz['ys'] 

# val_path = chemprop_dir / "tests" / "data" / "regression" / "rxn" / "e2sn2" / "fold_4" / "aam_val.csv"
# val_npz = np.load(f'../chemprop/data/super/e2sn2/fold_4/e2sn2_aam_val_processed_data.npz', allow_pickle=True)
# val_v = val_npz['node_attrs']
# val_e = val_npz['edge_attrs']
# val_idx_g = val_npz['edge_indices']
# val_y = val_npz['ys'] 

# test_path = chemprop_dir / "tests" / "data" / "regression" / "rxn" / "e2sn2" / "fold_4" / "aam_test.csv"
# test_npz = np.load(f'../chemprop/data/super/e2sn2/fold_4/e2sn2_aam_test_processed_data.npz', allow_pickle=True)
# test_v = test_npz['node_attrs']
# test_e = test_npz['edge_attrs']
# test_idx_g = test_npz['edge_indices']
# test_y = test_npz['ys'] 

In [ ]:
train_path = chemprop_dir / "tests" / "data" / "regression" / "rxn" / "barriers_cycloadd" / "train.csv"
train_npz = np.load(f'../chemprop/data/super/barriers_cycloadd/barriers_cycloadd_Gact_aam_train_processed_data.npz', allow_pickle=True)
train_v = train_npz['node_attrs']
train_e = train_npz['edge_attrs']
train_idx_g = train_npz['edge_indices']
train_y = train_npz['ys'] 

val_path = chemprop_dir / "tests" / "data" / "regression" / "rxn" / "barriers_cycloadd" / "val.csv"
val_npz = np.load(f'../chemprop/data/super/barriers_cycloadd/barriers_cycloadd_Gact_aam_val_processed_data.npz', allow_pickle=True)
val_v = val_npz['node_attrs']
val_e = val_npz['edge_attrs']
val_idx_g = val_npz['edge_indices']
val_y = val_npz['ys'] 

test_path = chemprop_dir / "tests" / "data" / "regression" / "rxn" / "barriers_cycloadd" / "test.csv"
test_npz = np.load(f'../chemprop/data/super/barriers_cycloadd/barriers_cycloadd_Gact_aam_test_processed_data.npz', allow_pickle=True)
test_v = test_npz['node_attrs']
test_e = test_npz['edge_attrs']
test_idx_g = test_npz['edge_indices']
test_y = test_npz['ys'] 

FileNotFoundError: [Errno 2] No such file or directory: '../chemprop/data/super/barriers_cycloadd/barriers_cycloadd_G_act_aam_train_processed_data.npz'

In [ ]:
print(train_idx_g.shape, val_y.shape, test_y.shape)

(4216,) (527,) (526,)


## Perform data splitting for training, validation, and testing

## Get ReactionDatasets

In [ ]:
train_dset = data.ReactionDataset(train_v, train_e, train_idx_g, train_y)
print(train_dset[0][3])
scaler = train_dset.normalize_targets()
# print(scaler)
print(train_dset[0][3])

val_dset = data.ReactionDataset(val_v, val_e, val_idx_g, val_y)
val_dset.normalize_targets(scaler)
test_dset = data.ReactionDataset(test_v, test_e, test_idx_g, test_y)

[5.84979105]
[[  5.84979105]
 [-44.51964951]
 [ -4.15103102]
 ...
 [ -6.86291885]
 [-40.39235687]
 [ -7.71586227]]
[[ 1.52293379]
 [-0.81575755]
 [ 1.05858804]
 ...
 [ 0.93267303]
 [-0.62412422]
 [ 0.89307022]]
[1.52293379]
[[-8.10949014e-02]
 [-2.40323370e-01]
 [ 4.67506943e-01]
 [-1.39443903e+00]
 [-6.59896824e-01]
 [ 3.24813367e-02]
 [-1.13267469e-01]
 [-8.97691414e-02]
 [ 3.47378885e-01]
 [ 1.79617783e-01]
 [-5.41389674e-01]
 [ 8.86066500e-01]
 [ 2.00428325e-01]
 [-9.16497153e-01]
 [ 1.33187548e+00]
 [ 1.34193994e+00]
 [ 6.23963672e-01]
 [-1.30115544e+00]
 [-1.84803647e-01]
 [-9.37277674e-01]
 [-7.84962072e-02]
 [-1.56933210e+00]
 [-5.74354840e-02]
 [ 5.22317304e-01]
 [ 1.08009518e-01]
 [-9.08019870e-01]
 [-6.87861834e-01]
 [ 3.78299928e-01]
 [ 1.16657977e+00]
 [-7.64645171e-01]
 [ 6.53532010e-01]
 [ 1.98043174e+00]
 [ 9.32548136e-01]
 [-2.51022615e-01]
 [-5.82192113e-01]
 [ 9.97022014e-01]
 [-7.17437390e-01]
 [ 1.39161792e+00]
 [-3.57409138e-01]
 [ 9.14855779e-01]
 [ 9.65291647e-0

In [ ]:
train_dset[0][3]

array([1.52293379])

In [ ]:
edge_index=train_dset[1][0][-2]
print(f'edge_index: {edge_index}')
reverse_index=train_dset[1][0][-1]
print(f'reverse_index: {reverse_index}')

edge_index: [[ 0  1  1  2  1  3  3  4  3  8  3 20  4  5  5  6  5 19  6  7  8  9  8 13
   9 10 10 11 11 12 12 13 14 15 15 16 16 17 16 19 16 26 17 18 19 20 20 21
  21 22 21 23 23 24 24 25 25 26 27 28 27 29 29 30 29 34 29 36 30 31 31 32
  31 35 32 33 35 36 37 27 37 28 37 29 37 30 37 31 37 32 37 33 37 34 37 35
  37 36]
 [ 1  0  2  1  3  1  4  3  8  3 20  3  5  4  6  5 19  5  7  6  9  8 13  8
  10  9 11 10 12 11 13 12 15 14 16 15 17 16 19 16 26 16 18 17 20 19 21 20
  22 21 23 21 24 23 25 24 26 25 28 27 29 27 30 29 34 29 36 29 31 30 32 31
  35 31 33 32 36 35 27 37 28 37 29 37 30 37 31 37 32 37 33 37 34 37 35 37
  36 37]]
reverse_index: [ 1  0  3  2  5  4  7  6  9  8 11 10 13 12 15 14 17 16 19 18 21 20 23 22
 25 24 27 26 29 28 31 30 33 32 35 34 37 36 39 38 41 40 43 42 45 44 47 46
 49 48 51 50 53 52 55 54 57 56 59 58 61 60 63 62 65 64 67 66 69 68 71 70
 73 72 75 74 77 76 79 78 81 80 83 82 85 84 87 86 89 88 91 90 93 92 95 94
 97 96]


In [ ]:
import numpy as np

np.arange(6).reshape(-1,2)[:, ::-1].ravel()

array([1, 0, 3, 2, 5, 4])

## Get dataloaders

In [ ]:
train_loader = data.build_dataloader(train_dset, num_workers=num_workers)
val_loader = data.build_dataloader(val_dset, num_workers=num_workers, shuffle=False)
test_loader = data.build_dataloader(test_dset, num_workers=num_workers, shuffle=False)

# Change Message-Passing Neural Network (MPNN) inputs here

## Message passing

Message passing blocks must be given the shape of the featurizer's outputs.

Options are `mp = nn.BondMessagePassing()` or `mp = nn.AtomMessagePassing()`

In [ ]:
train_v[0].shape[1]

23

In [ ]:
fdims = (train_v[0].shape[1],train_e[0].shape[1]) # the dimensions of the featurizer, given as (atom_dims, bond_dims).
mp = nn.BondMessagePassing(*fdims)

In [ ]:
print(*fdims)

23 6


## Aggregation

In [ ]:
print(nn.agg.AggregationRegistry)

ClassRegistry {
    'mean': <class 'chemprop.nn.agg.MeanAggregation'>,
    'sum': <class 'chemprop.nn.agg.SumAggregation'>,
    'norm': <class 'chemprop.nn.agg.NormAggregation'>
}


In [ ]:
# agg = nn.MeanAggregation()  #try Mean or Sum
agg = nn.SumAggregation()  #try Mean or Sum

## Feed-Forward Network (FFN)

In [ ]:
print(nn.PredictorRegistry)

ClassRegistry {
    'regression': <class 'chemprop.nn.predictors.RegressionFFN'>,
    'regression-mve': <class 'chemprop.nn.predictors.MveFFN'>,
    'regression-evidential': <class 'chemprop.nn.predictors.EvidentialFFN'>,
    'regression-quantile': <class 'chemprop.nn.predictors.QuantileFFN'>,
    'classification': <class 'chemprop.nn.predictors.BinaryClassificationFFN'>,
    'classification-dirichlet': <class 'chemprop.nn.predictors.BinaryDirichletFFN'>,
    'multiclass': <class 'chemprop.nn.predictors.MulticlassClassificationFFN'>,
    'multiclass-dirichlet': <class 'chemprop.nn.predictors.MulticlassDirichletFFN'>,
    'spectral': <class 'chemprop.nn.predictors.SpectralFFN'>
}


In [ ]:
output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)

In [ ]:
ffn = nn.RegressionFFN(output_transform=output_transform)

## Batch norm

In [ ]:
batch_norm = True

## Metrics

In [ ]:
print(nn.metrics.MetricRegistry)

ClassRegistry {
    'mse': <class 'chemprop.nn.metrics.MSE'>,
    'mae': <class 'chemprop.nn.metrics.MAE'>,
    'rmse': <class 'chemprop.nn.metrics.RMSE'>,
    'bounded-mse': <class 'chemprop.nn.metrics.BoundedMSE'>,
    'bounded-mae': <class 'chemprop.nn.metrics.BoundedMAE'>,
    'bounded-rmse': <class 'chemprop.nn.metrics.BoundedRMSE'>,
    'r2': <class 'chemprop.nn.metrics.R2Score'>,
    'binary-mcc': <class 'chemprop.nn.metrics.BinaryMCCMetric'>,
    'multiclass-mcc': <class 'chemprop.nn.metrics.MulticlassMCCMetric'>,
    'roc': <class 'chemprop.nn.metrics.BinaryAUROC'>,
    'prc': <class 'chemprop.nn.metrics.BinaryAUPRC'>,
    'accuracy': <class 'chemprop.nn.metrics.BinaryAccuracy'>,
    'f1': <class 'chemprop.nn.metrics.BinaryF1Score'>
}


In [ ]:
metric_list = [nn.metrics.RMSE(), nn.metrics.MAE()] 
# Only the first metric is used for training and early stopping

## Construct MPNN

In [ ]:
mpnn = models.MPNN(mp, agg, ffn, batch_norm, metric_list)
mpnn

MPNN(
  (message_passing): BondMessagePassing(
    (W_i): Linear(in_features=29, out_features=300, bias=False)
    (W_h): Linear(in_features=300, out_features=300, bias=False)
    (W_o): Linear(in_features=323, out_features=300, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (tau): ReLU()
    (V_d_transform): Identity()
    (graph_transform): Identity()
  )
  (agg): SumAggregation()
  (bn): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (predictor): RegressionFFN(
    (ffn): MLP(
      (0): Sequential(
        (0): Linear(in_features=300, out_features=300, bias=True)
      )
      (1): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.0, inplace=False)
        (2): Linear(in_features=300, out_features=1, bias=True)
      )
    )
    (criterion): MSE(task_weights=[[1.0]])
    (output_transform): UnscaleTransform()
  )
  (X_d_transform): Identity()
  (metrics): ModuleList(
    (0): RMSE(task_weights=[[1.0]])
    (1): MAE(task_weight

# Training and testing

## Set up trainer

In [ ]:
trainer = pl.Trainer(
    logger=False,
    enable_checkpointing=True,  # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=200,  # number of epochs to train for
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


## Start training

In [ ]:
trainer.fit(mpnn, train_loader, val_loader)

/home/labhhc2/anaconda3/envs/chemprop/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/labhhc2/Documents/workspace/D20/Tam/backup/chemprop_1/examples/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/home/labhhc2/anaconda3/envs/chemprop/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.

  | Name            | Type               | Params | Mode 
---------------------------------------------------------------
0 | message_passing | BondMessagePassing | 195 K  | train
1 | agg             | SumAggregation     | 0      | train
2 | bn              | BatchNorm1d        | 600    | train
3 | predictor     

/home/labhhc2/anaconda3/envs/chemprop/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Epoch 99: 100%|██████████| 66/66 [00:00<00:00, 125.23it/s, train_loss_step=0.0344, val_loss=0.0307, train_loss_epoch=0.0227] 

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 66/66 [00:00<00:00, 79.21it/s, train_loss_step=0.0344, val_loss=0.0307, train_loss_epoch=0.0227] 


## Test results

In [ ]:
results = trainer.test(mpnn, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/labhhc2/anaconda3/envs/chemprop/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 9/9 [00:00<00:00, 355.27it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test/mae          │     2.753438711166382     │
│         test/rmse         │    3.7457027435302734     │
└───────────────────────────┴───────────────────────────┘

In [ ]:
results

[{'test/rmse': 3.7457027435302734, 'test/mae': 2.753438711166382}]